In [85]:
import numpy as np
np.set_printoptions(linewidth = 200, precision = 4, suppress = True)

In [5]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
data

array([[ 1.    , -0.0461,  0.2312, ...,  0.2704,  0.4664,  0.3672],
       [-0.0461,  1.    , -0.0671, ..., -0.0515, -0.0944, -0.0349],
       [ 0.2312, -0.0671,  1.    , ...,  0.147 ,  0.2608,  0.6313],
       ...,
       [ 0.2704, -0.0515,  0.147 , ...,  1.    ,  0.2066,  0.1538],
       [ 0.4664, -0.0944,  0.2608, ...,  0.2066,  1.    ,  0.3486],
       [ 0.3672, -0.0349,  0.6313, ...,  0.1538,  0.3486,  1.    ]])

In [6]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [7]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [9]:
# One lower bound impl, and two upper bound impls
bounds[:, 60]

array([4.5265, 6.1343, 7.    ])

In [10]:
selected = np.where(nodes[:, 60] == 1)[0]
selected

array([57, 82, 83, 84, 85])

In [45]:
selected = np.r_[selected, [86]]
selected

array([57, 82, 83, 84, 85, 86])

In [46]:
data[selected, :][:, selected]

array([[1.    , 0.4014, 0.3903, 0.3946, 0.3465, 0.3374],
       [0.4014, 1.    , 0.9643, 0.9311, 0.95  , 0.0515],
       [0.3903, 0.9643, 1.    , 0.9818, 0.9886, 0.0701],
       [0.3946, 0.9311, 0.9818, 1.    , 0.9772, 0.0973],
       [0.3465, 0.95  , 0.9886, 0.9772, 1.    , 0.0642],
       [0.3374, 0.0515, 0.0701, 0.0973, 0.0642, 1.    ]])

In [47]:
np.sqrt(np.linalg.eigvalsh(data[selected, :][:, selected]))

array([0.0894, 0.1366, 0.2687, 0.7712, 1.0972, 2.0255])

In [48]:
# Use "raw" data SVD from here on out, singular values are slightly different.
np.random.seed(1)
raw_data = np.random.multivariate_normal(
    np.zeros_like(selected, float),
    data[selected, :][:, selected],
    size=100)
raw_data.shape

(100, 6)

In [49]:
np.linalg.svd(raw_data)[1]

array([19.7849, 12.0553,  7.1674,  2.6202,  1.3461,  0.862 ])

In [50]:
np.linalg.svd(np.c_[raw_data, raw_data])[1][::-1] / np.sqrt(2)

array([ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.862 ,  1.3461,  2.6202,  7.1674, 12.0553, 19.7849])

In [52]:
v = np.linalg.svd(raw_data[:, 0:5])[2][0:1, :].T
v

array([[0.2481],
       [0.4778],
       [0.4927],
       [0.4757],
       [0.4911]])

In [97]:
import scipy.linalg
mat = scipy.linalg.block_diag(v @ v.T, [[0]], np.eye(5) - (v @ v.T), [[1]])
rotation_matrix_small = scipy.linalg.block_diag(
    np.linalg.svd(raw_data[:, 0:5])[2].T,
    [[1]])
# mat = mat @ scipy.linalg.block_diag(rotation_matrix_small, rotation_matrix_small)

In [98]:
np.linalg.svd(np.c_[raw_data, raw_data] @ mat)[1][::-1]

array([ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.862 ,  1.3461,  2.6202,  7.1674, 12.0553, 19.7849])

In [104]:
# Gershgorin circle theorem after projecting onto two different matrices.
# We sparsified data_expanded, but Gershgorin bound is performing poorly!
apply_mat = np.c_[raw_data, raw_data] @ mat
data_expanded = apply_mat.T @ apply_mat
np.sort(np.sqrt(abs(data_expanded).sum(axis=1)))

array([ 0.    ,  4.7577,  4.8509,  5.0322,  5.6708, 13.7641, 14.8316, 17.4746, 20.5379, 20.5842, 20.8684, 20.9025])

In [103]:
# Gershgorin circle theorem (no change of variables).
np.sort(np.sqrt(abs(raw_data.T @ raw_data).sum(axis=1)))

array([16.0563, 17.6847, 20.2925, 20.3243, 20.5031, 20.6218])

In [105]:
data_expanded

array([[ 23.329 ,  44.9357,  46.3362,  44.7337,  46.1852,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  14.4556],
       [ 44.9357,  86.5538,  89.2514,  86.1647,  88.9605,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  27.8439],
       [ 46.3362,  89.2514,  92.033 ,  88.8502,  91.7331,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  28.7117],
       [ 44.7337,  86.1647,  88.8502,  85.7774,  88.5606,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  27.7187],
       [ 46.1852,  88.9605,  91.7331,  88.5606,  91.4341,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  28.6181],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ],
       [ -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ,  94.6169,  -9.6804, -11.6543, -10.2479, -16.7561,  46.4953],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,  -9.6804,   5.8146,   0.5809,